In [ ]:
import tensorflow as tf
import dlib
import cv2
import os
import numpy as np
from PIL import Image, ImageChops, ImageEnhance
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from statistics import mode

input_shape = (128, 128, 3)
pr_data = []
detector = dlib.get_frontal_face_detector()
y=[]

model = load_model('/content/drive/My Drive/project/deepfake_model.h5')

from flask import *

app = Flask(__name__)


@app.route('/')
def upload():
    return render_template("file.html")


@app.route('/success', methods=['POST'])
def success():
    if request.method == 'POST':
        f=request.files['file']
        f.save(f.filename)
        k='/content/'+f.filename
        cap = cv2.VideoCapture(k)
        frameRate = cap.get(5)
        while cap.isOpened():
            frameId = cap.get(1)
            ret, frame = cap.read()
            if ret != True:
                break
            if frameId % ((int(frameRate) + 1) * 1) == 0:
                face_rects, scores, idx = detector.run(frame, 0)
                for i, d in enumerate(face_rects):
                    x1 = d.left()
                    y1 = d.top()
                    x2 = d.right()
                    y2 = d.bottom()
                    crop_img = frame[y1:y2, x1:x2]
                    data = img_to_array(cv2.resize(crop_img, (128, 128))).flatten() / 255.0
                    data = data.reshape(-1, 128, 128, 3)
                    y.append(model.predict_classes(data))
        temp = [int(i) for i in y]
        templ = mode(temp)
        print(temp)
        if templ==0:
          g="fake video"
        else:
          g="real video"
        return render_template("success.html", name=g)

import portpicker
port = portpicker.pick_unused_port()
from google.colab import output
output.serve_kernel_port_as_window(port)

from gevent.pywsgi import WSGIServer
host='localhost'
app_server = WSGIServer((host, port), app)
app_server.serve_forever()

if __name__ == '__main__':
    app.run(debug=True,use_reloader=False)

<IPython.core.display.Javascript object>

127.0.0.1 - - [2020-10-04 09:23:28] "GET / HTTP/1.1" 200 365 0.002100
127.0.0.1 - - [2020-10-04 09:23:29] "GET /favicon.ico HTTP/1.1" 404 356 0.000739


[1, 1, 1, 1, 1, 1]


127.0.0.1 - - [2020-10-04 09:25:16] "POST /success HTTP/1.1" 200 247 58.348630
127.0.0.1 - - [2020-10-04 09:25:17] "GET /favicon.ico HTTP/1.1" 404 356 0.000685


# New Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
